In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.onnx

from src.nvidia_cnn import NvidiaCnn
from src.train_nvidia_cnn import train

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [7]:
model = NvidiaCnn()
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss()
epochs = 10

In [ ]:
batch_size = 32
train_loader = "TODO"
val_loader = "TODO"

In [ ]:
train(model, train_loader, val_loader, optimizer, criterion, epochs, device)

In [ ]:
x = torch.randn(1, 3, 224, 224, requires_grad=True)

torch.onnx.export(model,
                  x,
                  "model.onnx",
                  export_params=True,
                  opset_version=11)